# 如何串联可运行组件 How to chain runnables 

关于LangChain表达式语言（LCEL）的一个要点是，任何两个可运行组件都可以通过"串联"形成序列。前一个可运行组件的.invoke()调用的输出会作为下一个可运行组件的输入。这可以通过使用管道操作符（|）或更明确的.pipe()方法来实现，二者效果相同。  

生成的RunnableSequence自身也是一个可运行组件，这意味着它可以像任何其他可运行组件一样被调用、流式处理或进一步串联。

## 管道运算符：|
为了展示其工作原理，我们来看一个例子。我们将介绍 LangChain 中的一种常见模式：使用提示模板将输入格式化为聊天模型，最后使用输出解析器将聊天消息输出转换为字符串。

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")

chain = prompt | model | StrOutputParser()

提示和模型都是可运行的，并且提示调用的输出类型与聊天模型的输入类型相同，因此我们可以将它们链接在一起。然后，我们可以像任何其他可运行程序一样调用结果序列：

In [7]:
chain.invoke({"topic":"熊"})

'好的，这里有一个关于熊的笑话：\n\n三只熊坐在森林里的一棵大树下，它们分别是熊爸爸、熊妈妈和熊宝宝。突然，一只小鸟飞了过来，落在了它们的面前。\n\n熊爸爸看着小鸟说：“哇，好可爱的小鸟啊！我们可以把它带回家养吗？”\n\n熊妈妈摇了摇头说：“不行，亲爱的，我们不能随便抓小动物回家。我们应该让它自由地生活在大自然中。”\n\n熊宝宝听了爸爸妈妈的话，点了点头表示同意。然后，它站了起来，对着小鸟说：“好吧，小鸟，你可以走了。但是，在我们放你走之前，你能不能先告诉我，你为什么飞得那么快？”\n\n小鸟回答说：“因为我不想被你们这些笨重的家伙踩到！”'

## 强制
我们甚至可以将此链与更多可运行程序组合以创建另一个链。这可能涉及使用其他类型的可运行程序进行一些输入/输出格式化，具体取决于链组件所需的输入和输出。

例如，假设我们想将笑话生成链与另一个评估生成的笑话是否有趣的链组合在一起。

我们需要小心如何格式化下一个链的输入。在下面的例子中，链中的字典被自动解析并转换为 RunnableParallel，它并行运行其所有值并返回包含结果的字典。

这恰好是下一个提示模板期望的相同格式。它的实际效果如下：

In [11]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("{joke} 这是一个有趣的笑话吗？")

composed_chain = {"joke":chain} | analysis_prompt | model | StrOutputParser()

composed_chain.invoke({"topic": "熊"})

'是的，这是一个有趣的笑话。它利用了北极熊和企鹅生活在不同地理位置的事实，创造了一个幽默的情景。'

函数也将被强制转换为可运行对象，因此您也可以向链中添加自定义逻辑。以下链产生与之前相同的逻辑流程：

In [15]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_lambda.invoke({"topic": "白菜"})

'这个说法是一个幽默的玩笑，利用了“白菜”和“白来”在中文里的谐音，以及人们通常不会将动物与人类的行为或担忧联系在一起的荒诞感来制造笑点。实际上，小白兔并不会因为吃了卷心菜而变成“白菜”，这种表述显然是非现实的，因此构成了笑话的效果。'

## .pipe() 方法
我们还可以使用 .pipe() 方法组成相同的序列。如下所示：

In [16]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = RunnableParallel({"joke":chain}).pipe(analysis_prompt).pipe(model).pipe(StrOutputParser())

composed_chain_with_pipe.invoke({"topic":"狗"})

'是的，这是一个有趣的笑话。它用一种幽默的方式告诉我们，有时候我们不必刻意去追求幸福，而是应该专注于自己的生活和成长，幸福自然会伴随而来。'

In [17]:
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)